## Analysis scripts for investigating EKE in moist GCM simulations

Generates Figures 5-10, A1 in Lutsko et al (2024) "Atmospheric Moisture Decreases Mid-Latitude Eddy Kinetic Energy". All simulations were run with [Isca](https://execlim.github.io/IscaWebsite/), using the standard gray radiation/Frierson model set-up and varying the parameter ``es0`` in the constants nml.

## Preliminaries

Load modules, define constants, etc.

In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as ss

In [ ]:
#various constants and other things we need
cp = 1005.
g = 9.8
kappa = 2. / 7.
p0 = 100000.
e_r = 6.371 * 10 ** 6 
R = 287.
L = 2.5 * 10. ** 6

In [ ]:
files = ['/data/nlutsko/FMS/moisture/dry/atmos_daily.nc', '/data/nlutsko/FMS/moisture/es_0.001/atmos_daily.nc','/data/nlutsko/FMS/moisture/es_0.01/atmos_daily.nc','/data/nlutsko/FMS/moisture/es_0.1/atmos_daily.nc','/data/nlutsko/FMS/moisture/es_0.2/atmos_daily.nc','/data/nlutsko/FMS/moisture/es_0.4/atmos_daily.nc', '/data/nlutsko/FMS/moisture/es_0.6/atmos_daily.nc', '/data/nlutsko/FMS/moisture/es_0.8/atmos_daily.nc', '/data/nlutsko/FMS/moisture/control/atmos_daily.nc']
filesm = ['/data/nlutsko/FMS/moisture/dry/atmos_monthly.nc', '/data/nlutsko/FMS/moisture/es_0.001/atmos_monthly.nc','/data/nlutsko/FMS/moisture/es_0.01/atmos_monthly.nc','/data/nlutsko/FMS/moisture/es_0.1/atmos_monthly.nc','/data/nlutsko/FMS/moisture/es_0.2/atmos_monthly.nc','/data/nlutsko/FMS/moisture/es_0.4/atmos_monthly.nc', '/data/nlutsko/FMS/moisture/es_0.6/atmos_monthly.nc', '/data/nlutsko/FMS/moisture/es_0.8/atmos_monthly.nc','/data/nlutsko/FMS/moisture/control/atmos_monthly.nc']
f = len(files)
#Use monthly data wherever possible

#saturation fraction multiple
es = [0., 0.001, 0.01, 0.1, 0.2, 0.4, 0.6, 0.8, 1.]

## MAPE and EKE

Start by calculating MAPE and EKE

EKE is:
$$
EKE = \frac{1}{2g}\int_0^{p_s} (<u'^2> + <v'^2>)dp.
$$
In practice, integrate to where EKE within 20% of its max value to avoid differences in stratosphere.

MAPE is:
$$
MAPE = \frac{c_p}{2g}\int_0^{p_s} <\Gamma\left(\frac{p_0}{p}\right)^\kappa\left(\frac{\theta'}{\bar{\theta}}\right)^2> dp
$$

Approximate MAPE can either take near-surface average or go up to tropopause, do both:
$$
MAPE = \frac{c_p}{24g}<\bar{p}_s - \bar{p}_t> \Gamma<\partial_y\bar{\theta}>^2L_z^2
$$

Baroclinic zone = +/-15 degrees of max vertically-integrated PT flux, or where near-surface flux >= 30% of max. Seems to work best to do >=50% of max

In [ ]:
l = -2 #integrate to 900hPa
ehfl = -4 #EHF level
thresh = 0.5 #threshold for defining baroclinic zone. Here set to 50%

lim = 500 #exclude first 500 days of daily data
lim2 = 17 #exclude first 17 months of monthly data

In [ ]:
#functions for taking averages over baroclinic zones
def merid_mean(data, l1, l2, rad_lat):
    cos_lat = np.cos(rad_lat)
    ndata = np.trapz(data[:, l1:l2] * cos_lat[l1:l2], rad_lat[l1:l2], axis = 1)
    
    return ndata / np.trapz(cos_lat[l1:l2], rad_lat[l1:l2])

def merid_mean_1d(data, l1, l2, rad_lat):
    #for 1d data
    cos_lat = np.cos(rad_lat)
    ndata = np.trapz(data[l1:l2] * cos_lat[l1:l2], rad_lat[l1:l2])
    
    return ndata / np.trapz(cos_lat[l1:l2], rad_lat[l1:l2])

In [ ]:
#arrays for storing data
mape = np.zeros( f ) #Lorenz MAPE
mape2 = np.zeros( f ) #full troposphere approximation
mape3 = np.zeros( f ) #near-surface approximate
global_mape = np.zeros( f )
near_global_mape = np.zeros( f )

eke = np.zeros( f )
eke2 = np.zeros( f )
eke3 = np.zeros( f )

#energy cycle terms
ceape = np.zeros( f )
ceke = np.zeros( f )
czke = np.zeros( f )
ckk = np.zeros( f )
#precip contributions:
zPT = np.zeros( f )
ePT = np.zeros( f )

#and individual terms in MAPE:
Lz = np.zeros( f ) #baroclinic zone width
apt_ps = np.zeros( f ) #depth of troposphere
agamma = np.zeros( f ) #inverse stability
adptdy = np.zeros( f ) #meridional temperature gradient

#near-surface versions
agamma2 = np.zeros( f ) 
adptdy2 = np.zeros( f )

In [ ]:
#values so I didn't have to run things again
Lz = [4984203.38741995, 4828445.36589947, 4828445.36589947, 5295719.51526337, 5295719.51526337, 5295711.97968755, 5139953.92906867, 5139953.92906867, 4984195.90754818]
l1 = [80, 81, 81, 78, 78, 79, 80, 80, 81]
l2 = [112, 112, 112, 112, 112, 113, 113, 113, 113]

eke = [987867.3036008694, 989187.9060342757, 965300.2034222975, 926951.2226223493, 916903.560749129, 872111.6048195268, 843104.2509847481, 818816.7976677035, 809069.4086856389]
eke2 = [1405083.7576878201, 1405166.829447559, 1410277.243332272, 1467637.1251164149, 1446978.4048657184, 1327400.8182924206, 1242101.331792119, 1187973.868905095, 1151462.436085949]
eke3 = [1347365.0941855274, 1345778.7404621376, 1350073.0522996716, 1350083.0522996716, 1330739.7189100978, 1226147.7314380629, 1154997.611381712, 1106590.0037935367, 1074137.3634161488]

czke = [1.4722279269907808, 1.393764671079548, 0.4599090148130927, -0.9949150053678011, -1.3594298181984494, -1.9366255284337586, -1.8093309412407927, -1.8207329333744533, -1.4611473106985242]
ceape = [8.214524166981096, 7.9643811648739495, 7.891324083954635, 3.66732864254267, 2.411412063767874, 1.604835112421876, 1.2671926082841658, 1.046143682916829, 0.9260706356631225  ]
ceke = [13.374956267121453, 12.647457642116034, 11.494538971686739, 6.959252347207199, 5.3236581667457425, 4.170685363392416, 3.5921832392719004, 3.1962159444740936, 2.9855926758775277]
ckk = [-0.7749166792954344, -0.5800562964271028, -0.46740665810610205, -0.49772339482046624, -0.3872506252023321, -0.42316893170496456, -0.37187181241548734, -0.31075522766104574, -0.21811643022805918]

zPT = [0., 0.00010438855618524652, 0.002125889150299157, 0.020726106134716357, 0.01650060430565539,0.009992352284396022, 0.0063088730665606385, 0.0034548909520208037 , 0.002776947569570451 ]
ePT = [0., -0.00015707333460637544, -0.001243754525637672, -0.005026006810633346, -0.004758121848055831, -0.003342340736092011, -0.0024454332806561835, -0.001734614278542975, -0.0012581547326716467]



In [ ]:
0.003347563647765115 -0.0002532465289357158

In [ ]:
for z in range( f ):
    print("Doing:", z)
    print("Load data")
    #ds = xr.open_dataset(files[z])
    ds2 = xr.open_dataset(filesm[z]) #use monthly data wherever possible to speed things up
    
    if z == 0:
        lat = np.array(ds2.lat[:])
        rad_lat = np.array(ds2.lat[:]) * np.pi / 180.
        cos_lat = np.cos(rad_lat)
        cos_lat_2d = np.expand_dims(cos_lat, 0)
        press = np.array(ds2.pfull[:]) #approximate pressures for plotting

        d1 = len(press)
        d2 = len(rad_lat)

        nbk = (np.array(ds2.bk[1:]) + np.array(ds2.bk[:-1]))/2. #for converting from sigma to pressure coordinates

        #Things we need:
        zpot_temp = np.zeros( ( ( f, d1, d2)))
        zpressure = np.zeros( ( ( f, d1, d2)))
        
        j_min = np.zeros( f ).astype(int) #tropopause levels
        #l1 = np.zeros( f ).astype(int) #equatorward edge of baroclinic zone
        #l2 = np.zeros( f ).astype(int) #poleward edge of baroclinic zone

    print("Make potential temperature:")
    pressure = np.array(ds2.ps)[lim2:, np.newaxis] * nbk[np.newaxis, :, np.newaxis, np.newaxis] 
    pot_temp = np.array(ds2.temp[lim2:]) * (np.array(ds2.ps)[lim2:, np.newaxis] / pressure) ** kappa

    print("Now take zonal- and time-means")
    zpot_temp[z] = np.mean(np.mean( pot_temp, axis = 3), axis = 0)
    zpressure[z] = np.mean(np.mean(pressure, axis = 3), axis = 0)
    if z == 3:
        #for some reason, file is corrupted and it didn't save first two latitude points
        nh = zpot_temp[z, :, d2 // 2:]
        zpot_temp[z, :, :d2 // 2] = nh[:, ::-1]
        nh_p = zpressure[z, :, d2 // 2:]
        zpressure[z, :, :d2 // 2] = nh_p[:, ::-1]
 
    print("Tropopause heights")
    #where \gamma < -2K/km, interpolate to find pressure
    #can use monthly data here
    temp = np.mean(np.mean(np.array(ds2.temp[lim2:, :, :]), axis = 3), axis = 0)
    height = np.mean(np.mean(np.array(ds2.height[lim2:, :, :]), axis = 3), axis = 0)

    dT_dz = np.zeros( ( d1, d2))
    for i in range( d2):
        dT_dz[:, i] = np.gradient(temp[:, i], height[:, i]) * 1000.

    j_trop = np.zeros( d2 )
    p_trop = np.zeros( d2)
    for i in range( d2):
        for j in range( d1):
            if dT_dz[j, i] < -2.:
                c = (-2. - dT_dz[j - 1, i]) / ( dT_dz[j, i] -  dT_dz[j - 1, i])
                p_trop[i] = c * zpressure[z, j - 1, i] + (1. - c) * zpressure[z, j, i]
                j_trop[i] = j
                break
                
    ps = np.mean(np.mean(np.array(ds2.ps[lim2:, :]), axis = 2), axis = 0)
    pt_ps = ps - p_trop
    
    j_min[z] = int(max(j_trop[:])) #OGS08b suggests taking lowest level to exclude stratosphere
    """
    print("Baroclinic zone")
    #need daily \theta for eddy fluxes
    ev = ds.vcomp[lim:, ehfl] - np.expand_dims(np.mean(ds.vcomp[lim:, ehfl], axis = 2), 2)
    pressure = np.array(ds.ps)[lim:, np.newaxis] * nbk[np.newaxis, :, np.newaxis, np.newaxis] 
    pot_temp = np.array(ds.temp[lim:]) * (np.array(ds.ps)[lim:, np.newaxis] / pressure) ** kappa
    ept = pot_temp[:, ehfl] - np.expand_dims(np.mean(pot_temp[:, ehfl], axis = 2), 2)
    ev_pt = np.mean( np.mean( ev * ept, axis = 2), axis = 0) * cos_lat

    ls = np.where(ev_pt >= thresh * max(ev_pt[64:]))
    l1[z] = int(min(ls[0]))
    l2[z] = int(max(ls[0]))

    lat1 = rad_lat[l1[z]]
    lat2 = rad_lat[l2[z]]

    Lz[z] = e_r * (lat2 - lat1)
    """
    apt_ps[z] = merid_mean_1d(pt_ps, l1[z], l2[z], rad_lat)

Now calculate EKE and MAPEs

In [ ]:
#First do Lorenz MAPE
for z in range( f ):
    print("Average pressure")
    nzpress = merid_mean(zpressure[z], l1[z], l2[z], rad_lat)

    print("Calculate gamma")
    adpt_dp = np.zeros( ( d1, d2 ) )
    for i in range(d2):
        adpt_dp[:, i] = np.gradient(zpot_temp[z, :, i], zpressure[z, :, i] )  
    adpt_dp2 = merid_mean(adpt_dp, l1[z], l2[z], rad_lat)
    gamma = -kappa / nzpress /  adpt_dp2
    
    print("theta^2")
    mzpot_temp = merid_mean(zpot_temp[z], l1[z], l2[z], rad_lat)
    mzpot_temp2 = merid_mean(zpot_temp[z] ** 2, l1[z], l2[z], rad_lat)

    pt = mzpot_temp2 - mzpot_temp ** 2
    mape[z] = cp / 2. / g * np.trapz( gamma[j_min[z]:-2] * (p0 / nzpress[j_min[z]:-2]) ** kappa * pt[j_min[z]:-2], nzpress[j_min[z]:-2])
    print(mape[z])


In [ ]:
#Global MAPE
for z in range( f ):
    print("Average pressure")
    nzpress = merid_mean(zpressure[z, :, :], 0, d2, rad_lat)

    print("Calculate gamma")
    adpt_dp = np.zeros( ( d1, d2 ) )
    for i in range(d2):
        adpt_dp[:, i] = np.gradient(zpot_temp[z, :, i], zpressure[z, :, i] )  
    adpt_dp2 = merid_mean(adpt_dp, 0, d2, rad_lat)
    gamma = -kappa / nzpress /  adpt_dp2
    
    print("theta^2")
    mzpot_temp = merid_mean(zpot_temp[z], 0, d2, rad_lat)
    mzpot_temp2 = merid_mean(zpot_temp[z] ** 2, 0, d2, rad_lat)
    
    pt = mzpot_temp2 - mzpot_temp ** 2

    global_mape[z] = cp / 2. / g * np.trapz( gamma[j_min[z]:-2] * (p0 / nzpress[j_min[z]:-2]) ** kappa * pt[j_min[z]:-2], nzpress[j_min[z]:-2])
    
    print(global_mape[z])

In [ ]:
#near-Global MAPE
for z in range( f ):
    print("Average pressure")
    nzpress = merid_mean(zpressure[z], l1[z], l2[z], rad_lat)

    print("Calculate gamma")
    adpt_dp = np.zeros( ( d1, d2 ) )
    for i in range(d2):
        adpt_dp[:, i] = np.gradient(zpot_temp[z, :, i], zpressure[z, :, i] )  
    adpt_dp2 = merid_mean(adpt_dp, 71, d2, rad_lat)
    gamma = -kappa / nzpress /  adpt_dp2
    
    print("theta^2")
    mzpot_temp = merid_mean(zpot_temp[z], 71, d2, rad_lat)
    mzpot_temp2 = merid_mean(zpot_temp[z] ** 2, 71, d2, rad_lat)

    pt = mzpot_temp2 - mzpot_temp ** 2
    near_global_mape[z] = cp / 2. / g * np.trapz( gamma[j_min[z]:-2] * (p0 / nzpress[j_min[z]:-2]) ** kappa * pt[j_min[z]:-2], nzpress[j_min[z]:-2])
    
    print(near_global_mape[z])

In [ ]:
#Now full troposphere integration
for z in range( f ):
    print("Average pressure")
    nzpress = merid_mean(zpressure[z], l1[z], l2[z], rad_lat)
    
    print("Calculate gamma") 
    adpt_dp = np.zeros( ( d1, d2 ) )
    for i in range(d2):
        adpt_dp[:, i] = np.gradient(zpot_temp[z, :, i], zpressure[z, :, i] )  
    adpt_dp2 = merid_mean(adpt_dp, l1[z], l2[z], rad_lat)

    gamma = -kappa / nzpress /  adpt_dp2
    agamma[z] = np.trapz(gamma[j_min[z]:l], nzpress[j_min[z]:l]) / (max(nzpress[j_min[z]:l]) - min(nzpress[j_min[z]:l]))#np.mean(gamma[j_min[z]:l])
    
    print("Calculate temperature gradients") 
    dptdy = np.gradient( zpot_temp[z], rad_lat, axis = 1)  / e_r #O'Gorman uses T, not sure why
    dptdy2  = merid_mean(dptdy, l1[z], l2[z], rad_lat)
    adptdy[z] = np.mean(dptdy2[j_min[z]:l])
   
    mape2[z] = cp / 24. / g * apt_ps[z] * agamma[z] * adptdy[z] ** 2 * Lz[z] ** 2
    print(mape2[z])

In [ ]:
#near surface MAPE
f1 = -7
f2 = -4
mape4 = np.zeros(f)
for z in range( f ):
    print("Calculate temperature gradients") 
    dptdy = np.gradient( zpot_temp[z], rad_lat, axis = 1)  / e_r #O'Gorman uses T, not sure why
    dptdy2 = merid_mean(dptdy, l1[z], l2[z], rad_lat)
    adptdy2[z] = np.mean(dptdy2[f1:f2])

    print("Calculate gamma")
    adpt_dp = np.zeros( ( d1, d2 ) )
    for i in range(d2):
        adpt_dp[:, i] = np.gradient(zpot_temp[z, :, i], zpressure[z, :, i] ) 
    adpt_dp2 = merid_mean(adpt_dp, l1[z], l2[z], rad_lat)
    gamma = -kappa / nzpress /  adpt_dp2
    agamma2[z] = np.mean(gamma[f1:f2])
    
    mape3[z] = cp / 24. / g * apt_ps[z] * agamma2[z]  * adptdy2[z] **2   * Lz[z] ** 2
    print(mape3[z])

In [ ]:
#EKE
for z in range( 2, 3 ):
    print("Load data")
    ds = xr.open_dataset("/home/nlutsko/Isca_data/frierson/run0001/atmos_daily.nc")#files[z])
    
    print("EKE")
    eu = ds.ucomp[lim:, :, :] - ds.ucomp[lim:, :, :].mean(dim="lon")
    ev = ds.vcomp[lim:, :, :] - ds.vcomp[lim:, :, :].mean(dim="lon")
    
    eu2 = eu ** 2
    ev2 = ev ** 2
    zm_eke = eu2.mean(dim="lon").mean(dim="time") + ev2.mean(dim="lon").mean(dim="time")
    
    nzpress = merid_mean(zpressure[z], l1[z], l2[z], rad_lat)
    eke_m = merid_mean(zm_eke, l1[z], l2[z], rad_lat)
    ls = np.where(eke_m > 0.2 * np.max(eke_m)) #since EKE extends to quite different heights, only go from where >20% downwards
    ls = int(np.min(ls))
    eke[z] = np.trapz(eke_m[ls:], nzpress[ls:]) / g / 2.
    print("EKE", eke[z])
    
    eke2[z] = np.trapz(np.trapz(zm_eke * np.expand_dims(cos_lat[:], 0), rad_lat[:]), press * 100) / g / 2.
    eke3[z] = np.trapz(np.trapz(zm_eke[:, 71:] * np.expand_dims(cos_lat[71:], 0), rad_lat[71:]), press * 100) / g 
    #don't need factor 2 as just taking one hemisphere
    print("Global EKEs", eke2[z], eke3[z])

Finally we can plot Figure 5:

In [ ]:
#colors for plotting
cs = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),  
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),  
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),  
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),  
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]  

# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.  
for i in range(len(cs)):  
    r, gr, b = cs[i]  
    cs[i] = (r / 255., gr / 255., b / 255.)

In [ ]:
fig = plt.figure( figsize = (10, 7) )
fig.subplots_adjust(left =0.1, bottom = 0.1, top = 0.93, hspace = 0.4, wspace = 0.35, right = 0.97)

ax = plt.subplot(2, 2, 1)
plt.title("a) EKE", fontsize = 18, loc = "left")

plt.plot(es, np.array(eke)/ 10. ** 6, 'kx-', linewidth = 0.5, markersize = 8 )
plt.xlim([-0.05, 1.05])
plt.ylim([0.8, 1.0])

plt.xlabel("$\gamma$", fontsize = 18)
plt.ylabel("[MJ/m$^2$]", fontsize = 18)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.yticks([0.8, 0.85, 0.9, 0.95, 1.], fontsize = 16)
plt.xticks(fontsize = 16)

ax = plt.subplot(2, 2, 2)
plt.title("b) MAPE", fontsize = 18, loc = "left")

plt.plot(es, np.array(mape)/ 10. ** 6, 'kx-', linewidth = 0.5, markersize = 8 )
plt.plot(es, np.array(mape2)/ 10. ** 6, 'ro-', linewidth = 0.5 , markersize = 8 )
plt.plot(es, np.array(mape3)/ 10. ** 6 , 'bv-', linewidth = 0.5, markersize = 8 )

plt.legend(["MAPE", "approx MAPE", "near-surface MAPE"], fontsize = 15)

plt.xlim([-0.05, 1.05])
plt.ylim([1., 16.])

plt.xlabel("$\gamma$", fontsize = 18)
plt.ylabel("[MJ/m$^2$]", fontsize = 18)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.yticks([3., 6., 9., 12, 15.], fontsize = 16)
plt.xticks(fontsize = 16)

ax = plt.subplot(2, 2, 3)
plt.title("c)", fontsize = 18, loc = "left")

plt.plot(np.array(mape[:])/ 10. ** 6 , np.array(eke)/ 10. ** 6, 'kx-', markersize = 8 )
#Check other fits
#plt.plot(np.array(mape2)/ 10. ** 6 , np.array(eke)/ 10. ** 6, 'ro-', linewidth = 0.5 )
#plt.plot(np.array(mape3)/ 10. ** 6, np.array(eke)/ 10. ** 6, 'bv-', linewidth = 0.5 )

plt.xlabel("MAPE [MJ/m$^2$]", fontsize = 18)
plt.ylabel("EKE [MJ/m$^2$]", fontsize = 18)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.xlim([1., 12.])
plt.ylim([0.8, 1.0])
plt.yticks([0.8, 0.85, 0.9, 0.95, 1.], fontsize = 16)
plt.xticks([3., 6., 9., 12], fontsize = 16)

plt.text(1.4, 0.98, "moist", fontsize = 16)
plt.text(3.8, 0.98, "dry", fontsize = 16)

s1, i1 = ss.linregress(mape[4:], eke[4:])[:2]
s2, i2 = ss.linregress(mape[:5], eke[:5])[:2]

x1 = np.arange(1., 5., 1.)
x2 = np.arange(2., 13., 1.)

plt.plot(x1, i1 / 10. ** 6 + s1 * x1, color = cs[6], linewidth = 0.75)
plt.plot(x2, i2 / 10. ** 6 + s2 * x2, color = cs[6], linewidth = 0.75)


ax = plt.subplot(2, 2, 4)
plt.title("d) contributions to MAPE", fontsize = 18, loc = "left")

plt.plot(es, apt_ps / apt_ps[f - 1] * 100., 'ko-', linewidth = 0.5, markersize = 8 )
plt.plot(es, agamma2 / agamma2[f - 1] * 100., 'kv-', linewidth = 0.5, markersize = 8 )
plt.plot(es, (adptdy2 ** 2) / (adptdy2[f - 1] ** 2) * 100., 'kD-', linewidth = 0.5, markersize = 8 )
plt.plot(es, (np.asarray(Lz) ** 2) / (Lz[f - 1] ** 2) * 100., 'kx-', linewidth = 0.5, markersize = 8 )

plt.xlabel("$\gamma$", fontsize = 18)
plt.ylabel("% change", fontsize = 18)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.yticks(fontsize = 16)
plt.xticks(fontsize = 16)

plt.ylim([0., 450.])
plt.axhline(y = 100., color = 'k', linewidth = 0.5)
plt.legend(["$\Delta p$", "$\Gamma$", "$(\partial \\theta/\partial y)^2$", "$L_z^2$"], ncol = 2, fontsize = 15)

plt.savefig("images/FMS_EKE.png")
plt.savefig("images/FMS_EKE.pdf")

Plot potential temperatures (Figure 6)

In [ ]:
fig = plt.figure( figsize = (16, 6) )
fig.subplots_adjust(left =0.1, bottom = 0.1, top = 0.95, hspace = 0.25, wspace = 0.1, right = 0.95)

titles = ["dry", "$\gamma$ = 0.2", "$\gamma$ = 0.4", "$\gamma$ = 0.6", "$\gamma$ = 0.8", "$\gamma$ = 1"]
a = [0, 4, 5, 6, 7, 8]
for z in range( 6 ):
    ax = plt.subplot(2, 3, z + 1)
    plt.title(titles[z], loc = "left", fontsize = 18)
    levels = np.arange(250., 350., 5.)

    im = plt.contourf(ds2.lat[:], ds2.pfull[:],zpot_temp[a[z], :, :], levels, cmap = plt.cm.RdBu_r, extend = "max")
    plt.ylim([970., 0.])  
    plt.xticks([-60., -30., 0., 30., 60.], fontsize = 16)

    if z > 2:
        plt.xlabel("latitude", fontsize = 18)
        plt.xticks([-60., -30., 0., 30., 60.], fontsize = 16)
    else:
        plt.xticks([-60., -30., 0., 30., 60.])
        ax.axes.xaxis.set_ticklabels([])

    if z == 0 or z == 3:
        plt.ylabel("pressure [hPa]", fontsize = 18)
        plt.yticks([800., 600., 400., 200.], fontsize = 16)
    else:
        plt.yticks([800., 600., 400., 200.])
        ax.axes.yaxis.set_ticklabels([])
        
fig.subplots_adjust(right = 0.89)

cbar_ax = fig.add_axes([0.91, 0.22, 0.01, 0.6])
cb = fig.colorbar(im, cax=cbar_ax)
cb.set_label( "[K]", fontsize = 16  ) 

plt.savefig("images/FMS_isentropes.png")
plt.savefig("images/FMS_isentropes.pdf")

Check global EKEs (Figure A1):

In [ ]:
fig = plt.figure( figsize = (10, 4) )
fig.subplots_adjust(left =0.1, bottom = 0.2, top = 0.93, hspace = 0.4, wspace = 0.3, right = 0.97)

ax = plt.subplot(1, 2, 1)
plt.title("a) EKE", fontsize = 18, loc = "left")

plt.plot(es, np.array(eke2) / 10. ** 6 * 2., 'kx-', linewidth = 0.5, markersize = 8 )
plt.plot(es, np.array(eke3) / 10. ** 6 * 2., 'ro-', linewidth = 0.5, markersize = 8 )

plt.legend(["global EKE", "near-global EKE"], fontsize = 15)

plt.xlim([-0.05, 1.05])
plt.ylim([2., 3.])

plt.xlabel("$\gamma$", fontsize = 18)
plt.ylabel("[MJ/m$^2$]", fontsize = 18)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.yticks([2., 2.2, 2.4, 2.6, 2.8, 3.], fontsize = 16)
plt.xticks(fontsize = 16)

ax = plt.subplot(1, 2, 2)
plt.title("b) MAPE", fontsize = 18, loc = "left")

plt.plot(es, np.array(global_mape)/ 10. ** 6, 'kx-', linewidth = 0.5, markersize = 8 )
plt.plot(es, np.array(near_global_mape)/ 10. ** 6, 'ro-', linewidth = 0.5, markersize = 8 )


plt.xlim([-0.05, 1.05])
plt.ylim([1., 22.])

plt.xlabel("$\gamma$", fontsize = 18)
plt.ylabel("[MJ/m$^2$]", fontsize = 18)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.yticks([5., 10., 15., 20.], fontsize = 16)
plt.xticks(fontsize = 16)

plt.savefig("images/FMS_globalEKE.png")
plt.savefig("images/FMS_globalEKE.pdf")

## Lorenz Energy Cycle

Terms:
$$
C_{EAPE} = \frac{-R}{g}\int_0^p \left(\frac{p_0}{p}\right)^\kappa <\left([T^*v^*]\frac{\partial}{\partial y} + [T^*\omega^*]\frac{\partial}{\partial p}\right)(\Gamma T') >dp
$$

$$
C_{EKE} = \frac{-R}{g}\int_0^p p^{-1}<T^*\omega^*>dp
$$

$$
C_{ZKE} = \frac{-R}{g}\int_0^p p^{-1}<[T'][\omega']>dp
$$

In [ ]:
#ceape and ceke
for z in range( f ):
    print("Load data")
    ds = xr.open_dataset(files[z])

    print("First calculate zonal-means")
    omegaz = np.mean(np.mean(ds.omega[lim:, :, :], axis = 3), axis = 0)
    vz = np.mean(np.mean(ds.vcomp[lim:, :, :], axis = 3), axis = 0)
    Tz = np.mean(np.mean(ds.temp[lim:, :, :], axis = 3), axis = 0)
    
    print("Calculate area means")
    Ta = merid_mean(Tz, l1[z], l2[z], rad_lat)
    
    print("Now calculate zonal-mean deviations from area mean")
    Td = Tz - np.expand_dims(Ta, 1)
    
    print("Eddies")
    omegae = ds.omega[lim:, :, :] - np.expand_dims(np.expand_dims( omegaz, 0), 3)
    Te = ds.temp[lim:, :, :] - np.expand_dims(np.expand_dims( Tz, 0), 3)
    
    oeTe = np.mean( np.mean(omegae * Te, axis = 3), axis = 0)
    
    omegae = []
    ve = ds.vcomp[lim:, :, :] - np.expand_dims(np.expand_dims( vz, 0), 3)    
    veTe = np.mean( np.mean(ve * Te, axis = 3), axis = 0)
    ve = []
    Te = []
        
    print("Calculate gamma") 
    nzpress = merid_mean(zpressure[z], l1[z], l2[z], rad_lat)
    adpt_dp = np.zeros( ( d1, d2 ) )
    for i in range(d2):
        adpt_dp[:, i] = np.gradient(zpot_temp[z, :, i], zpressure[z, :, i] )  
    #adpt_dp2 = merid_mean(adpt_dp, l1[z], l2[z], rad_lat)

    gamma = -kappa / np.expand_dims(nzpress, 1) /  adpt_dp
    
    print("Take derivatives") 
    dT_dp = np.zeros( ( d1, d2 ) )
    for i in range(d2):
        dT_dp[:, i] = np.gradient(Td[:, i] * gamma[:, i], zpressure[z, :, i] )  
    dT_dy = np.gradient(Td * gamma, rad_lat, axis = 1) / e_r
    
    average = veTe * dT_dy + oeTe * dT_dp
    average = merid_mean(average, l1[z], l2[z], rad_lat)
    average2 = merid_mean(oeTe, l1[z], l2[z], rad_lat)

    ceape[z] = -R / g * np.trapz(((p0 / nzpress[j_min[z]:l]) ** kappa) * gamma[j_min[z]:l] * average[j_min[z]:l], nzpress[j_min[z]:l] )
    ceke[z] = -R / g * np.trapz(average2[j_min[z]:l] / nzpress[j_min[z]:l], nzpress[j_min[z]:l] )
    print(ceape[z], ceke[z])

In [ ]:
#now do czke
#can use monthly data for this one
for z in range( f ):
    print("Load data")
    ds = xr.open_dataset(filesm[z])
    print("First calculate zonal-means")
    omegaz = np.mean(np.mean(ds.omega[lim2:, :, :], axis = 3), axis = 0)
    Tz = np.mean(np.mean(ds.temp[lim2:, :, :], axis = 3), axis = 0)

    nzpress = merid_mean(zpressure[z], l1[z], l2[z], rad_lat)
  
    print("Calculate area means")
    omegaa = merid_mean(omegaz, l1[z], l2[z], rad_lat)
    Ta = merid_mean(Tz, l1[z], l2[z], rad_lat)
    
    print("Now calculate zonal-mean deviations from area mean")
    omegad = omegaz - np.expand_dims(omegaa, 1)
    Td = Tz - np.expand_dims(Ta, 1)
   
    odTd = merid_mean(omegad[:, :] * Td[:, :], l1[z], l2[z], rad_lat)
    czke[z] = -R / g * np.trapz(odTd[j_min[z]:l] / nzpress[j_min[z]:l], nzpress[j_min[z]:l] )

    print(czke[z])

Precipitation terms:

$$
G_{Z} = \frac{-1}{g}\int_0^{ps} \left(\frac{p_s}{p}\right)^\kappa\Gamma <T^*Q_L^*> dp, \\
G_{E} = \frac{-1}{g}\int_0^{ps} \left(\frac{p_s}{p}\right)^\kappa\Gamma <T'Q_L'> dp,
$$
where $Q_L$ is temperature tendency due to condensation times $c_p$.

In [ ]:
#zPT and ePT
for z in range( 2, 3 ):
    print("Load data")
    #ds = xr.open_dataset(files[z])
    ds = xr.open_dataset("/home/nlutsko/Isca_data/frierson/run0001/atmos_daily.nc")#files[z])

    print("First calculate area means:")
    Qz = np.mean(np.mean(ds.dt_tg_condensation[lim:, :, :], axis = 3), axis = 0)
    Tz = np.mean(np.mean(ds.temp[lim:, :, :], axis = 3), axis = 0)

    Qa = merid_mean(Qz, l1[z], l2[z], rad_lat)
    Ta = merid_mean(Tz, l1[z], l2[z], rad_lat)

    print("Now calculate zonal-mean deviations from area mean")
    Qd = Qz - np.expand_dims(Qa, 1)
    Td = Tz - np.expand_dims(Ta, 1)

    print("Eddies:")
    Qe = ds.dt_tg_condensation[lim:] - Qz
    Te = ds.temp[lim:]  -  Tz
    TeQe = np.mean( np.mean(Qe * Te, axis = 0), axis = 2) #* cos_lat_2d ** 2
  
    print("Calculate gamma") 
    nzpress = merid_mean(zpressure[z], l1[z], l2[z], rad_lat)
    adpt_dp = np.zeros( ( d1, d2 ) )
    for i in range(d2):
        adpt_dp[:, i] = np.gradient(zpot_temp[z, :, i], zpressure[z, :, i] )  
    adpt_dp2 = merid_mean(adpt_dp, l1[z], l2[z], rad_lat)

    gamma = -kappa / nzpress /  adpt_dp2
    
    average = merid_mean( Qd * Td, l1[z], l2[z], rad_lat)
    average2 = merid_mean(TeQe, l1[z], l2[z], rad_lat)

    zPT[z] = 1. / g * np.trapz(((p0 / nzpress[j_min[z]:l]) ** kappa) * gamma[j_min[z]:l] * average[j_min[z]:l], nzpress[j_min[z]:l] )
    ePT[z] = 1. / g * np.trapz(((p0 / nzpress[j_min[z]:l]) ** kappa) * gamma[j_min[z]:l] * average2[j_min[z]:l], nzpress[j_min[z]:l] )

    print(zPT[z], ePT[z])

In [ ]:
#Finally we can plot!
fig = plt.figure( figsize = (12, 4) )
fig.subplots_adjust(left =0.1, bottom = 0.2, top = 0.85, wspace = 0.2, right = 0.95)

ax = plt.subplot(1, 2, 1)
plt.title("a) MAPE budget", fontsize = 16, loc = "left")

plt.plot(es, -np.array(ceape), 'kD-', linewidth = 0.5, markersize = 8)
plt.plot(es, -np.array(czke), 'ko-', linewidth = 0.5, markersize = 8)
plt.plot(es, np.array(zPT) * cp , 'kx--', linewidth = 0.5, markersize = 8)
plt.plot(es, np.array(czke) + ceape - np.array(zPT) * cp, 'kv-', linewidth = 0.5, markersize = 8)


plt.legend(["-C(EAPE)", "C(ZKE)", "G$_{Z,P}$", "G$_{Z, NP}$"], ncol = 2, fontsize = 10, loc = "upper right")

plt.xlabel("$\gamma$", fontsize = 16)
plt.ylabel("tendency [Wm$^{-2}$]", fontsize = 16.)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.yticks([-10., 0., 10., 20.], fontsize = 14)
plt.xticks(fontsize = 14)

plt.ylim([-22., 24.])
plt.axhline(y = 0., color = 'k', linewidth = 0.5)

plt.axvline(x = 0.05, color = 'k', linestyle = ':', linewidth = 0.5)

ax = plt.subplot(1, 2, 2)
plt.title("b) EAPE budget", fontsize = 16, loc = "left")

plt.plot(es, np.array(ceape), 'kD-', linewidth = 0.5, markersize = 8)
plt.plot(es, -np.array(ceke), 'ko-', linewidth = 0.5, markersize = 8)
plt.plot(es, -np.array(ePT) * cp  , 'kx--', linewidth = 0.5, markersize = 8)
plt.plot(es, np.array(ePT) * cp - np.array(ceape) + np.array(ceke) , 'kv-', linewidth = 0.5, markersize = 8)

plt.legend(["C(EAPE)", "-C(EKE)", "G$_{E,P}$", "G$_{E, NP}$"], ncol = 2, fontsize = 10, loc = "lower right")

plt.xlabel("$\gamma$", fontsize = 16)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.yticks([-10., 0., 10., 20.], fontsize = 14)
plt.xticks(fontsize = 14)

plt.ylim([-15., 11.])
plt.axhline(y = 0., color = 'k', linewidth = 0.5)

plt.axvline(x = 0.05, color = 'k', linestyle = ':', linewidth = 0.5)
   
plt.savefig("FMS_Lorenz_Cycle.pdf")
plt.savefig("FMS_Lorenz_Cycle.png")

Now plot friction and effective exchange coefficients (Figure 10). The latter are backed out from the surface sensible heat flux. 

For the friction, calculate it as a residual from the EKE budget. So need $C_kk$:

$$
C_{kk} = \frac{-1}{g}\int_0^{ps} <([u^*v^*]\frac{\partial }{\partial y} + [u^*\omega^*]\frac{\partial }{\partial p})[u]>dp  - \frac{1}{g}\int_0^{ps} <([v^{*2}]\frac{\partial}{\partial y} + [v^*\omega^*]\frac{\partial}{\partial p} - \frac{tan\phi}{r}[u^*u^*])[v]>dp
$$

In [ ]:
for z in range( 1, f ):
    ds = xr.open_dataset(files[z])
    
    print("First calculate zonal-means")
    uz = np.mean(np.mean(ds.ucomp[lim:, :, :], axis = 3), axis = 0)
    vz = np.mean(np.mean(ds.vcomp[lim:, :, :], axis = 3), axis = 0)
    omegaz = np.mean(np.mean(ds.omega[lim:, :, :], axis = 3), axis = 0)
  
    print("Now calculate zonal-mean deviations from area mean")
    ue = ds.ucomp[lim:, :, :] - np.expand_dims(np.expand_dims( uz, 0), 3)
    ve = ds.vcomp[lim:, :, :] - np.expand_dims(np.expand_dims( vz, 0), 3)
    omegae = ds.omega[lim:, :, :] - np.expand_dims(np.expand_dims( omegaz, 0), 3)
     
    oeue = np.mean( np.mean(omegae * ue, axis = 3), axis = 0)
    ueue = np.mean( np.mean(ue ** 2, axis = 3), axis = 0)
    oeve = np.mean( np.mean(omegae * ve, axis = 3), axis = 0)
    veue = np.mean( np.mean(ve * ue, axis = 3), axis = 0)
    veve = np.mean( np.mean(ve ** 2, axis = 3), axis = 0)
    
    print("Take derivatives")
    du_dp = np.zeros( ( d1, d2 ) )
    dv_dp = np.zeros( ( d1, d2 ) )
    for i in range(d2):
        du_dp[:, i] = np.gradient(uz[:, i], zpressure[z, :, i] ) 
        dv_dp[:, i] = np.gradient(vz[:, i], zpressure[z, :, i] )      
    du_dy = np.gradient(uz, rad_lat, axis = 1) / e_r
    dv_dy = np.gradient(vz, rad_lat, axis = 1) / e_r
    
    if z == 0:
        tan_lat = np.tan(rad_lat)   
    average = veue * du_dy + oeue * du_dp + veve * dv_dy + oeve * dv_dp - tan_lat * ueue / e_r * vz
    average = merid_mean(average, l1[z], l2[z], rad_lat)

    nzpress = merid_mean(zpressure[z], l1[z], l2[z], rad_lat)
    ckk[z] = -1 / g * np.trapz(average[j_min[z]:l], nzpress[j_min[z]:l] )
    print(ckk[z])


In [ ]:
C = np.zeros((f, d2)) #effective exchange coefficients
for i in range(f):
    #Note that we need daily data to calculate winds
    print("Doing: ", i)
    if i != 2:
        ds = xr.open_dataset(files[i]) 
    else:
        ds = xr.open_dataset("/home/nlutsko/Isca_data/frierson/run0001/atmos_daily.nc")#files[z])
        
    sh = ds.flux_t.mean(dim="time")  
    v_abs = np.mean( np.sqrt(ds.ucomp[:, -1] ** 2 + ds.vcomp[:, -1] ** 2), axis = 0)
    ts = ds.t_surf.mean(dim="time")
    ta = np.mean( ds.temp[:, -1], axis = 0)
    
    Cd = sh / (ts - ta) / 1005. * v_abs
    C[i] = np.mean(Cd, axis = 1) / np.mean(v_abs, axis = 1) ** 2

In [ ]:
#for plotting
from matplotlib.colors import LinearSegmentedColormap

gamma= 1.
rels=  { 
  "pair2_a": [0.0117647061124444, 0.05098039284348488, 0.6352941393852234], 
  "pair2_b": [0.07058823853731155, 0.658823549747467, 0.07058823853731155], 
  "aug2": [0.9058823585510254, 0.5411764979362488, 0.03529411926865578], 
  "aug1_add": [0.0, 0.40392157435417175, 0.1882352977991104], 
  "aug2_add": [0.6666666865348816, 0.1411764770746231, 0.23137255012989044], 
  "grey": [0.4431372582912445, 0.4470588266849518, 0.4470588266849518], 
  "lead3": [0.07058823853731155, 0.658823549747467, 0.07058823853731155], 
  "lead2": [0.95686274766922, 0.1764705926179886, 0.12156862765550613], 
  "lead1": [0.07450980693101883, 0.3294117748737335, 0.6509804129600525], 
  "gridcolor": [0.8196078538894653, 0.8196078538894653, 0.8196078538894653], 
  "axiscolor": [0.11372549086809158, 0.11372549086809158, 0.11372549086809158], 
  "cascade3": [0.4470588266849518, 0.6196078658103943, 0.843137264251709], 
  "cascade2": [0.07450980693101883, 0.3294117748737335, 0.6509804129600525], 
  "plus": [0.95686274766922, 0.1764705926179886, 0.12156862765550613], 
  "aug1": [0.07058823853731155, 0.658823549747467, 0.07058823853731155], 
  "cascade1": [0.0117647061124444, 0.05098039284348488, 0.6352941393852234],
   "minus": [0.07450980693101883, 0.3294117748737335, 0.6509804129600525], 
   "cascade4": [0.7333333492279053, 0.7921568751335144, 0.8823529481887817], 
    "green": [0.05882352963089943, 0.545098066329956, 0.05882352963089943] ,
   "lightgreen": [144. /255., 238./255., 144./255.],
   "black": [0.11372549086809158, 0.11372549086809158, 0.11372549086809158]}

l = 9
cascade_r=LinearSegmentedColormap.from_list("my_colormap",
                                (rels['grey'], rels['cascade4'], rels['cascade3'], rels['cascade2'], rels['cascade1'], rels['green'] ), N=l, gamma=gamma)

evenly_spaced_interval = np.linspace(0., 1., l)
colors = [cascade_r(x) for x in evenly_spaced_interval]

In [ ]:
fig = plt.figure( figsize = (12, 4) )
fig.subplots_adjust(left =0.1, bottom = 0.2, top = 0.85, wspace = 0.2, right = 0.95)

ax = plt.subplot(1, 2, 1)
plt.title("a)", fontsize = 16, loc = "left")
#Calculate friction as a residual from EKE budget
plt.plot(es, np.array(ckk) - np.array(ceke), 'kx-', linewidth = 0.5)

plt.xlabel("$\gamma$", fontsize = 16)
plt.ylabel("eddy friction [Wm$^{-2}$]", fontsize = 16.)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.yticks([-12., -9., -6., -3.], fontsize = 14)
plt.xticks(fontsize = 14)

plt.ylim([-14., -1.])
plt.axhline(y = 0., color = 'k', linewidth = 0.5)

ax = plt.subplot(1, 2, 2)
plt.title("b)", fontsize = 16, loc = "left")

lat = np.linspace(-90., 90., 128)

evenly_spaced_interval = np.linspace(0., 1., f)
colors = [cascade_r(x) for x in evenly_spaced_interval]

for i in range( f):
        plt.plot(lat, C[i] * 1000., color = colors[i])

plt.xlabel("latitude", fontsize = 16)
plt.ylabel("estimated $C$ $\\times 10^{-3}$", fontsize = 16.)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.xlim([20., 70.])
plt.ylim([0.7, 1.4])

plt.text(43, 1.18, "dry", fontsize = 14)
plt.text(58, 0.77, "moist", fontsize = 14)

plt.xticks(fontsize = 14)
plt.yticks([0.8, 1., 1.2, 1.4], fontsize = 14)

plt.savefig("FMS_friction.pdf")
plt.savefig("FMS_friction.png")

## Other plots

Plot zonal-mean precip and surface turbulence fluxes (Figure 8)

In [ ]:
precip = np.zeros( ( f, 128))
lh = np.zeros( ( f, 128))
sh = np.zeros( ( f, 128))

for z in range( f ):
    ds = xr.open_dataset(filesm[z]) #use monthly data wherever possible to speed things up
    
    precip[z] = ds.precipitation.mean(dim="lon").mean(dim="time")
    lh[z] = ds.flux_lhe.mean(dim="lon").mean(dim="time")
    sh[z] = ds.flux_t.mean(dim="lon").mean(dim="time")
        

In [ ]:
fig = plt.figure( figsize = (8, 8) )
fig.subplots_adjust(left =0.15, bottom = 0.1, top = 0.9, hspace = 0.3, wspace = 0.1, right = 0.95)


ax = plt.subplot(3, 1, 1)
plt.title("a)", loc = "left", fontsize = 16)
for z in range( f ):   
    plt.plot( ds.lat, precip[z] * 86400., c = colors[z])
    
plt.ylabel("precipitation\n [mm/day]", fontsize = 14)
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.xlim([-88., 88.])
plt.xticks([-60., -30., 0., 30., 60.], fontsize = 0)
plt.yticks(fontsize = 12)

ax = plt.subplot(3, 1, 2)
plt.title("b)", loc = "left", fontsize = 16)

for z in range( f ):   
    plt.plot( ds.lat, lh[z], c = colors[z])
    
plt.ylabel("surface latent\n heat flux [Wm$^{-2}$]", fontsize= 14)
plt.ylim([-30., 300.])
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)
plt.xlim([-88., 88.])
plt.xticks([-60., -30., 0., 30., 60.], fontsize = 0)

plt.yticks(fontsize = 12)

ax = plt.subplot(3, 1, 3)
plt.title("c)", loc = "left", fontsize = 16)

for z in range( f ):   
    plt.plot( ds.lat, sh[z], c = colors[z])

plt.xlabel("latitude", fontsize = 14)
plt.ylabel("surface sensible\n heat flux [Wm$^{-2}$]", fontsize = 14)
plt.xlim([-88., 88.])

plt.ylim([-30., 300.])
plt.xticks([-60., -30., 0., 30., 60.], fontsize = 12)
plt.yticks(fontsize = 12)

ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

plt.axhline(y = 0, color = 'k', linewidth =0.5, linestyle = ':')
plt.text(-60, 130, "dry", fontsize = 14)
plt.text(-50, 10, "moist", fontsize = 14)

plt.savefig("surface_profiles.pdf")
plt.savefig("surface_profiles.png")

Plot zonal-mean anomalous latent heating (Figure 9)

In [ ]:
fig = plt.figure( figsize = (16, 6) )
fig.subplots_adjust(left =0.1, bottom = 0.1, top = 0.95, hspace = 0.25, wspace = 0.1, right = 0.95)

#show mid-latitude regions, these latitude bounds work well
l1 = 72
l2 = 110

titles = ["dry", "$\gamma$ = 0.2", "$\gamma$ = 0.4", "$\gamma$ = 0.6", "$\gamma$ = 0.8", "$\gamma$ = 1"]
for z in range( 6 ):
    print("Load data")
    print(filesm[a[z]])
    ds = xr.open_dataset(filesm[a[z]])

    print("Precip:")
    print("First calculate zonal-means")
    Qz = np.mean(np.mean(ds.dt_tg_condensation[lim2:, :, :], axis = 3), axis = 0)
    #Load temperature if want to plot QT 
    #Tz = np.mean(np.mean(ds.temp[lim2:, :, :], axis = 3), axis = 0)
    
    print("Calculate area means")
    Qa = merid_mean(Qz, l1, l2, rad_lat)
    #Ta = merid_mean(Tz, l1, l2, rad_lat)
   
    print("Now calculate zonal-mean deviations from area mean")
    Qd = Qz - np.expand_dims(Qa, 1)
    #Td = Tz - np.expand_dims(Ta, 1)

    levs = np.arange(-16., 17., 1.)
    ax = plt.subplot(2, 3, z + 1)
    plt.title(titles[z], loc = "left", fontsize = 18)
    
    Qd = np.ma.masked_where(abs(Qd * 86400.) < 0.1, Qd)
    im = plt.contourf(ds.lat[l1:l2], ds.pfull, Qd[:, l1:l2] * 86400., levs, cmap = plt.cm.RdBu_r, extend= "both")
    plt.ylim([970., 0.])  

    if z > 2:
        plt.xlabel("latitude", fontsize = 18)
        plt.xticks([30., 40., 50., 60.], fontsize = 16)
    else:
        plt.xticks([30., 40., 50., 60.], fontsize = 16)
        ax.axes.xaxis.set_ticklabels([])

    if z == 0 or z == 3:
        plt.ylabel("pressure [hPa]", fontsize = 18)
        plt.yticks([800., 600., 400., 200.], fontsize = 16)
    else:
        plt.yticks([800., 600., 400., 200.])
        ax.axes.yaxis.set_ticklabels([])
    plt.xlim([25., 65.])
    
fig.subplots_adjust(right = 0.89)

cbar_ax = fig.add_axes([0.91, 0.22, 0.01, 0.6])
cb = fig.colorbar(im, cax=cbar_ax)
cb.set_label( "[$Q^*_L$] [Kday$^{-1}$]", fontsize = 16  ) 

plt.savefig("FMS_anom_LH.pdf")
plt.savefig("FMS_anom_LH.png")    